In [63]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
# from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

In [64]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [65]:
df = pd.read_csv(r"H:\Nasa Space Apps 2025\data\cleaned_data.csv")
df.head()

,kepid,kepoi_name,koi_disposition,koi_pdisposition,koi_score,koi_period,koi_duration,koi_depth,koi_time0bk,koi_prad,koi_ror,koi_dor,koi_impact,koi_incl,koi_srad,koi_smass,koi_steff,koi_slogg,koi_smet,koi_kepmag,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_vet_stat,koi_vet_date,koi_disp_prov,koi_period_err1,koi_period_err2,koi_duration_err1,koi_duration_err2,koi_depth_err1,koi_depth_err2,koi_prad_err1,koi_prad_err2,koi_steff_err1,koi_steff_err2
0,10797460,K00752.01,CONFIRMED,CANDIDATE,1.000,9.488036,2.95750,615.8,170.538750,2.26,0.022344,24.810,0.146,89.66,0.927,0.919,5455.0,4.467,0.14,15.347,0,0,0,0,Done,2018-08-16,q1_q17_dr25_sup_koi,2.775000e-05,-2.775000e-05,0.08190,-0.08190,19.5,-19.5,0.26,-0.15,81.0,-81.0
1,10797460,K00752.02,CONFIRMED,CANDIDATE,0.969,54.418383,4.50700,874.8,162.513840,2.83,0.027954,77.900,0.586,89.57,0.927,0.919,5455.0,4.467,0.14,15.347,0,0,0,0,Done,2018-08-16,q1_q17_dr25_sup_koi,2.479000e-04,-2.479000e-04,0.11600,-0.11600,35.5,-35.5,0.32,-0.19,81.0,-81.0
2,10811496,K00753.01,CANDIDATE,CANDIDATE,0.000,19.899140,1.78220,10829.0,175.850252,14.60,0.154046,53.500,0.969,88.96,0.868,0.961,5853.0,4.544,-0.18,15.436,0,0,0,0,Done,2018-08-16,q1_q17_dr25_sup_koi,1.494000e-05,-1.494000e-05,0.03410,-0.03410,171.0,-171.0,3.92,-1.31,158.0,-176.0
3,10848459,K00754.01,FALSE POSITIVE,FALSE POSITIVE,0.000,1.736952,2.40641,8079.2,170.307565,33.46,0.387394,3.278,1.276,67.09,0.791,0.836,5805.0,4.564,-0.52,15.597,0,1,0,0,Done,2018-08-16,q1_q17_dr25_sup_koi,2.630000e-07,-2.630000e-07,0.00537,-0.00537,12.8,-12.8,8.50,-2.83,157.0,-174.0
4,10854555,K00755.01,CONFIRMED,CANDIDATE,1.000,2.525592,1.65450,603.3,171.595550,2.75,0.024064,8.750,0.701,85.41,1.046,1.095,6031.0,4.438,0.07,15.509,0,0,0,0,Done,2018-08-16,q1_q17_dr25_sup_koi,3.761000e-06,-3.761000e-06,0.04200,-0.04200,16.9,-16.9,0.88,-0.35,169.0,-211.0


In [66]:
df = df.drop(columns=['kepid', 'kepoi_name', 'koi_pdisposition', 'koi_vet_stat', 'koi_vet_date', 'koi_disp_prov'])
df.columns

Index(['koi_disposition', 'koi_score', 'koi_period', 'koi_duration',
       'koi_depth', 'koi_time0bk', 'koi_prad', 'koi_ror', 'koi_dor',
       'koi_impact', 'koi_incl', 'koi_srad', 'koi_smass', 'koi_steff',
       'koi_slogg', 'koi_smet', 'koi_kepmag', 'koi_fpflag_nt', 'koi_fpflag_ss',
       'koi_fpflag_co', 'koi_fpflag_ec', 'koi_period_err1', 'koi_period_err2',
       'koi_duration_err1', 'koi_duration_err2', 'koi_depth_err1',
       'koi_depth_err2', 'koi_prad_err1', 'koi_prad_err2', 'koi_steff_err1',
       'koi_steff_err2'],
      dtype='object')

In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9564 entries, 0 to 9563
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   koi_disposition    9564 non-null   object 
 1   koi_score          9564 non-null   float64
 2   koi_period         9564 non-null   float64
 3   koi_duration       9564 non-null   float64
 4   koi_depth          9564 non-null   float64
 5   koi_time0bk        9564 non-null   float64
 6   koi_prad           9564 non-null   float64
 7   koi_ror            9564 non-null   float64
 8   koi_dor            9564 non-null   float64
 9   koi_impact         9564 non-null   float64
 10  koi_incl           9564 non-null   float64
 11  koi_srad           9564 non-null   float64
 12  koi_smass          9564 non-null   float64
 13  koi_steff          9564 non-null   float64
 14  koi_slogg          9564 non-null   float64
 15  koi_smet           9564 non-null   float64
 16  koi_kepmag         9563 

In [68]:
X = df.drop(columns=['koi_disposition'])

scaler = RobustScaler()
X_scaled = scaler.fit_transform(X)

In [69]:
y = df['koi_disposition'].astype('category').cat.codes  # 0,1,2 labels

y[:5]

0    1
1    1
2    0
3    2
4    1
dtype: int8

In [70]:
X.shape, y.shape

((9564, 30), (9564,))

In [71]:
X_train, X_temp, y_train, y_temp = train_test_split(
    X_scaled, y, test_size=0.3, random_state=42, stratify=y
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

In [89]:
y_train = torch.tensor(y_train.values, dtype=torch.long)
y_val   = torch.tensor(y_val.values, dtype=torch.long)
y_test  = torch.tensor(y_test.values, dtype=torch.long)

y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_val_tensor   = torch.tensor(y_val, dtype=torch.long)
y_test_tensor  = torch.tensor(y_test, dtype=torch.long)

Tensor shapes:
X_train: torch.Size([6694, 30]), y_train: torch.Size([6694])
X_val: torch.Size([1435, 30]), y_val: torch.Size([1435])
X_test: torch.Size([1435, 30]), y_test: torch.Size([1435])
Classes: ['CANDIDATE' 'CONFIRMED' 'FALSE POSITIVE']


In [74]:
print("Train shape:", X_train_tensor.shape, y_train_tensor.shape)
print("Val shape:", X_val_tensor.shape, y_val_tensor.shape)
print("Test shape:", X_test_tensor.shape, y_test_tensor.shape)
print("Classes:", label_encoder.classes_)

Train shape: torch.Size([6694, 30]) torch.Size([6694])
Val shape: torch.Size([1435, 30]) torch.Size([1435])
Test shape: torch.Size([1435, 30]) torch.Size([1435])
Classes: ['CANDIDATE' 'CONFIRMED' 'FALSE POSITIVE']


In [75]:
batch_size = 64

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset   = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset  = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [91]:
class ExoplanetMLP(nn.Module):
    def __init__(self, input_dim, hidden1=256, hidden2=128, hidden3=64, output_dim=3, dropout=0.4):
        super(ExoplanetMLP, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden1),
            nn.BatchNorm1d(hidden1),
            nn.ReLU(),
            nn.Dropout(dropout),

            nn.Linear(hidden1, hidden2),
            nn.BatchNorm1d(hidden2),
            nn.ReLU(),
            nn.Dropout(dropout),

            nn.Linear(hidden2, hidden3),
            nn.BatchNorm1d(hidden3),
            nn.ReLU(),
            nn.Dropout(dropout),

            nn.Linear(hidden3, output_dim)  # final layer
        )
        
    def forward(self, x):
        return self.net(x)


In [97]:
input_dim = X_train_tensor.shape[1]
output_dim = len(label_encoder.classes_)
model = ExoplanetMLP(input_dim=input_dim, output_dim=3).to(device)

Model created with 190467 parameters
Input dim: 30, Output dim: 3
Classes: ['CANDIDATE' 'CONFIRMED' 'FALSE POSITIVE']


In [98]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5)

In [99]:
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler=None, epochs=50, patience=7):
    model = model.to(device)
    
    best_val_loss = float("inf")
    patience_counter = 0
    history = {"train_loss": [], "val_loss": []}
    best_model_wts = model.state_dict()

    for epoch in range(epochs):
        # ---- Training ----
        model.train()
        running_loss = 0.0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        epoch_train_loss = running_loss / len(train_loader)

        # ---- Validation ----
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                val_loss += loss.item()

        epoch_val_loss = val_loss / len(val_loader)

        history["train_loss"].append(epoch_train_loss)
        history["val_loss"].append(epoch_val_loss)

        if scheduler:
            scheduler.step(epoch_val_loss)

        print(f"Epoch [{epoch+1}/{epochs}] | Train Loss: {epoch_train_loss:.4f} | Val Loss: {epoch_val_loss:.4f}")

        # ---- Early Stopping ----
        if epoch_val_loss < best_val_loss:
            best_val_loss = epoch_val_loss
            patience_counter = 0
            best_model_wts = model.state_dict()
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered")
                break

    model.load_state_dict(best_model_wts)
    return model, history

In [100]:
model = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=50, patience=7)

Epoch [1/50] | Train Loss: 0.6958 | Train Acc: 0.6972 | Val Loss: 0.7747 | Val Acc: 0.5558
Epoch [5/50] | Train Loss: 0.3556 | Train Acc: 0.8567 | Val Loss: 0.3621 | Val Acc: 0.8286
Epoch [5/50] | Train Loss: 0.3556 | Train Acc: 0.8567 | Val Loss: 0.3621 | Val Acc: 0.8286
Epoch [10/50] | Train Loss: 0.3149 | Train Acc: 0.8760 | Val Loss: 1.4805 | Val Acc: 0.5077
Epoch [10/50] | Train Loss: 0.3149 | Train Acc: 0.8760 | Val Loss: 1.4805 | Val Acc: 0.5077
Epoch [15/50] | Train Loss: 0.2903 | Train Acc: 0.8927 | Val Loss: 0.8355 | Val Acc: 0.5106
Early stopping triggered at epoch 15
Best validation accuracy: 0.8556

Training completed! Best validation accuracy: 0.8556
Epoch [15/50] | Train Loss: 0.2903 | Train Acc: 0.8927 | Val Loss: 0.8355 | Val Acc: 0.5106
Early stopping triggered at epoch 15
Best validation accuracy: 0.8556

Training completed! Best validation accuracy: 0.8556


=== Model Performance ===
Train - Loss: 0.8873, Accuracy: 0.5093
Val   - Loss: 0.8735, Accuracy: 0.5106
Test  - Loss: 0.8836, Accuracy: 0.5073
